In [ ]:
!nvidia-smi
# !nvcc --version

Tue Nov 28 14:24:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# import numpy as np
# np.set_printoptions(precision=3, suppress=True)
# import pandas as pd
# import matplotlib.pyplot as plt
# import torch
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/content/drive') #, force_remount=True)
import sys
sys.path.insert(0,'/content/drive/Colab Notebooks/LLaVA-main')

MessageError: ignored

In [ ]:
# import torch
# import gc
# gc.collect()
# with torch.no_grad():
#     torch.cuda.empty_cache()

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/LLaVA-main
!pip install --upgrade pip
!pip install -e .

/content/drive/MyDrive/Colab Notebooks/LLaVA-main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/LLaVA-main
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 130.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
    

In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model
import os
import re
import json

model_path = "./llava-v1.5-13b/"
# model_path = "/content/drive/MyDrive/Colab Notebooks/LLaVA-main/llava-v1.5-13b/"
model_name = "llava-v1.5-13b"
tokenizer, llava_model, image_processor, context_len = load_pretrained_model(
    model_path, None, model_name
)
print("finish loading")
args = type('Args', (), {
    "conv_mode": None,
    "sep": ",",
    # "temperature": 0.2,
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

from llava.utils import disable_torch_init
disable_torch_init()
%cd /content/drive/MyDrive/Colab Notebooks/Visual ICL/

config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

finish loading
/content/drive/MyDrive/Colab Notebooks/Visual ICL


In [ ]:
prompt = "What are the things I should be cautious about when I visit here?"
image_file = "https://llava-vl.github.io/static/images/view.jpg"

print(eval_model(prompt, image_file, tokenizer, llava_model, image_processor, context_len, args))
prompt2 = '''
What is the object's color in this image? Answer from the following options:
(1)black (2)blue (3)green (4)pink (5)purple (6)red (7)white (8)yellow
'''
prompt3 = '''
What is the object in this image? Answer from the following options:
(1)flower (2)leaf (3)box (4)building (5)cup (6)bag (7)chair (8)car
'''
image_file2 = "/content/drive/MyDrive/Colab Notebooks/LLaVA-main/01.jpg"
print(eval_model(prompt2, image_file2, tokenizer, llava_model, image_processor, context_len, args))
eval_model(prompt3, image_file2, tokenizer, llava_model, image_processor, context_len, args)

When visiting the pier over the lake, there are a few things you should be cautious about to ensure a safe and enjoyable experience. First, be mindful of the weather conditions, as the image shows a cloudy day. Cloudy weather can sometimes lead to sudden changes in weather, such as rain or strong winds, which could make the pier slippery or even dangerous. Second, be aware of the water depth and any potential hazards, such as submerged rocks or debris, as the pier extends out into the lake. Lastly, be respectful of the surrounding environment and other visitors, and follow any posted rules or guidelines to ensure a pleasant experience for everyone.
(6) red


'(8) car'

# Artist

In [ ]:
def analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path):
    for mllm in mllms:
        for shot in shots:
            score_per_task = 0
            strict_score_per_task = 0
            number_per_task = 0
            samples_mean = []
            for theta in theta_list: # one function
                cur_path = get_path(mllm, shot, theta)
                score_per_object = 0
                strict_score_per_object = 0
                number_per_object = 0
                samples = []
                for filename in os.listdir(cur_path):
                    if filename.endswith(".jpg") or filename.endswith(".png"):
                        image_file = os.path.join(cur_path, filename)
                        file_name = os.path.splitext(filename)[0]
                        out1 = eval_model(prompt1, image_file, tokenizer, llava_model, image_processor, context_len, args)
                        out2 = eval_model(prompt2, image_file, tokenizer, llava_model, image_processor, context_len, args)
                        correct_caption = file_name.split('_')[-2] + ' ' + theta
                        correct_first = x_list.index(file_name.split('_')[-2]) + 1
                        correct_second = theta_list.index(theta) + 1
                        strict_score = 0
                        score = 0
                        first_number = re.search(r'\d', out1)
                        if first_number:
                            first_number = int(first_number.group(0))
                            if first_number == correct_first:
                                score += 0.5
                        else:
                            print('out1 error:', out1)
                            with open('./LLaVA_evaluation/error.txt', 'a') as f:
                                f.write('out1 error:' + out1 + '(' + filename + ')' + '\n')
                        secnond_number = re.search(r'\d', out2)
                        if secnond_number:
                            secnond_number = int(secnond_number.group(0))
                            if secnond_number == correct_second:
                                score += 0.5
                        else:
                            print('out2 error:', out2)
                            with open('./LLaVA_evaluation/error.txt', 'a') as f:
                                f.write('out2 error:' + out2 + '(' + filename + ')' + '\n')
                        if score >= 1:
                            strict_score += 1
                        sample = {
                            "name": filename,
                            "correct_caption": correct_caption,
                            "first_ans": out1,
                            "sec_ans": out2,
                            "correct_first": correct_first,
                            "correct_second": correct_second,
                            "score": score,
                            "strict_score": strict_score
                        }
                        print('[', number_per_object, ']', sample)
                        samples.append(sample)
                        score_per_object += score
                        strict_score_per_object += strict_score
                        number_per_object += 1
                sample_mean = {
                        "name": theta,
                        "score": score_per_object/number_per_object,
                        "strict_score": strict_score_per_object/number_per_object,
                        "number": number_per_object
                }
                print(sample_mean)
                print('--------------------------------')
                samples_mean.append(sample_mean)
                samples.append(sample_mean)
                score_per_task += score_per_object
                strict_score_per_task += strict_score_per_object
                number_per_task += number_per_object
                json_file = './LLaVA_evaluation_m/detail/' + task_name + '_' + mllm + '_' + shot + 'shot_' + theta + '(' + str(round(score_per_object/number_per_object,2)) + ')' + '(' + str(round(strict_score_per_object/number_per_object,2)) + ')' + '.json'
                with open(json_file, 'w') as f:
                    json.dump(samples, f, indent=4)
            sample_mean = {
                    "name": task_name,
                    "score": score_per_task/number_per_task,
                    "strict_score": strict_score_per_task/number_per_task,
                    "number": number_per_task
            }
            samples_mean.append(sample_mean)
            json_file = './LLaVA_evaluation_m/' + task_name + '_' + mllm + '_' + shot + 'shot_' + 'mean' + '(' + str(round(score_per_task/number_per_task,2)) + ')' + '(' + str(round(strict_score_per_task/number_per_task,2)) + ')' + '.json'
            with open(json_file, 'w') as f:
                json.dump(samples_mean, f, indent=4)

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
task_name = '[8]actiontoactionanimal'
mllms = ['seed', 'emu', 'gill']
shots = ['1', '2', '4']
x_list = ['wink', 'run', 'sing', 'fly', 'sit', 'drink', 'sleep', 'eat']
theta_list = ['dog', 'cat', 'pig', 'lion', 'bird', 'monkey', 'man', 'woman']
prompt1 = 'What is the object\'s action in this image? Answer from the following options: (1)wink (2)run (3)sing (4)fly (5)sit (6)drink (7)sleep (8)eat'
prompt2 = 'What is the object in this image? Answer from the following options: (1)dog (2)cat (3)pig (4)lion (5)bird (6)monkey (7)man (8)woman'

def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/action_animal/action_' + theta

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

Streaming output truncated to the last 5000 lines.
[ 196 ] {'name': 'sleep_eat_drink_result.jpg', 'correct_caption': 'drink pig', 'first_ans': '8', 'sec_ans': '1', 'correct_first': 6, 'correct_second': 3, 'score': 0, 'strict_score': 0}
[ 197 ] {'name': 'sing_wink_drink_result.jpg', 'correct_caption': 'drink pig', 'first_ans': '(1) wink', 'sec_ans': '(2) cat', 'correct_first': 6, 'correct_second': 3, 'score': 0, 'strict_score': 0}
[ 198 ] {'name': 'run_fly_sit_result.jpg', 'correct_caption': 'sit pig', 'first_ans': '(2) run', 'sec_ans': '(1) dog', 'correct_first': 5, 'correct_second': 3, 'score': 0, 'strict_score': 0}
[ 199 ] {'name': 'fly_run_drink_result.jpg', 'correct_caption': 'drink pig', 'first_ans': '(2) run', 'sec_ans': '(3) pig', 'correct_first': 6, 'correct_second': 3, 'score': 0.5, 'strict_score': 0}
[ 200 ] {'name': 'sing_sleep_sit_result.jpg', 'correct_caption': 'sit pig', 'first_ans': '(7) sleep', 'sec_ans': '(1) dog', 'correct_first': 5, 'correct_second': 3, 'score': 0, '

In [ ]:
task_name = '[9]animaltoactionanimal'
mllms = ['seed', 'emu', 'gill']
shots = ['1', '2', '4']
theta_list = ['wink', 'run', 'sing', 'fly', 'sit', 'drink', 'sleep', 'eat']
x_list = ['dog', 'cat', 'pig', 'lion', 'bird', 'monkey', 'man', 'woman']
prompt2 = 'What is the object\'s action in this image? Answer from the following options: (1)wink (2)run (3)sing (4)fly (5)sit (6)drink (7)sleep (8)eat'
prompt1 = 'What is the object in this image? Answer from the following options: (1)dog (2)cat (3)pig (4)lion (5)bird (6)monkey (7)man (8)woman'

def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/action_animal/' + theta + '_animal'

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

Streaming output truncated to the last 5000 lines.
[ 196 ] {'name': 'man_cat_bird_result.jpg', 'correct_caption': 'bird sing', 'first_ans': '8', 'sec_ans': '(5) sit', 'correct_first': 5, 'correct_second': 3, 'score': 0, 'strict_score': 0}
[ 197 ] {'name': 'lion_woman_dog_result.jpg', 'correct_caption': 'dog sing', 'first_ans': '8', 'sec_ans': '(3) sing', 'correct_first': 1, 'correct_second': 3, 'score': 0.5, 'strict_score': 0}
[ 198 ] {'name': 'woman_dog_lion_result.jpg', 'correct_caption': 'lion sing', 'first_ans': '8', 'sec_ans': '(3) sing', 'correct_first': 4, 'correct_second': 3, 'score': 0.5, 'strict_score': 0}
[ 199 ] {'name': 'pig_lion_monkey_result.jpg', 'correct_caption': 'monkey sing', 'first_ans': '(1) dog', 'sec_ans': '(1) wink', 'correct_first': 6, 'correct_second': 3, 'score': 0, 'strict_score': 0}
[ 200 ] {'name': 'dog_woman_bird_result.jpg', 'correct_caption': 'bird sing', 'first_ans': '8', 'sec_ans': '(3) sing', 'correct_first': 5, 'correct_second': 3, 'score': 0.5, 's

In [ ]:
task_name = '[10]backgroundtobackgroundobject'
mllms = ['seed', 'emu', 'gill']
shots = ['1', '2', '4']
x_list = ['beach', 'street', 'park', 'forest', 'office', 'classroom', 'gym', 'library']
theta_list = ['dog', 'cat', 'robot', 'car', 'chair', 'monkey', 'man', 'woman']
prompt1 = 'What is the background in this image? Answer from the following options: (1)beach (2)street (3)park (4)forest (5)office (6)classroom (7)gym (8)library'
prompt2 = 'What is the object in this image? Answer from the following options: (1)dog (2)cat (3)robot (4)car (5)chair (6)monkey (7)man (8)woman'

def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/background_object/background_' + theta

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

[ 0 ] {'name': 'office_forest_result.jpg', 'correct_caption': 'forest dog', 'first_ans': '5', 'sec_ans': '(1) dog', 'correct_first': 4, 'correct_second': 1, 'score': 0.5, 'strict_score': 0}
[ 1 ] {'name': 'classroom_office_result.jpg', 'correct_caption': 'office dog', 'first_ans': '5', 'sec_ans': '(8) woman', 'correct_first': 5, 'correct_second': 1, 'score': 0.5, 'strict_score': 0}
[ 2 ] {'name': 'library_office_result.jpg', 'correct_caption': 'office dog', 'first_ans': '8', 'sec_ans': '(1) dog', 'correct_first': 5, 'correct_second': 1, 'score': 0.5, 'strict_score': 0}
[ 3 ] {'name': 'park_gym_result.jpg', 'correct_caption': 'gym dog', 'first_ans': '(4) forest', 'sec_ans': '(1) dog', 'correct_first': 7, 'correct_second': 1, 'score': 0.5, 'strict_score': 0}
[ 4 ] {'name': 'library_beach_result.jpg', 'correct_caption': 'beach dog', 'first_ans': '8', 'sec_ans': '(5) chair', 'correct_first': 1, 'correct_second': 1, 'score': 0, 'strict_score': 0}
[ 5 ] {'name': 'classroom_gym_result.jpg', '

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[ 3 ] {'name': 'park_beach_result.jpg', 'correct_caption': 'beach monkey', 'first_ans': '(4) forest', 'sec_ans': '6', 'correct_first': 1, 'correct_second': 6, 'score': 0.5, 'strict_score': 0}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-4dd1b942db5f>", line 12, in <cell line: 12>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 20, in analysis
    out1 = eval_model(prompt1, image_file, tokenizer, llava_model, image_processor, context_len, args)
  File "/content/drive/MyDrive/Colab Notebooks/LLaVA-main/llava/eval/run_llava.py", line 101, in eval_model
  File "/content/drive/MyDrive/Colab Notebooks/LLaVA-main/llava/eval/run_llava.py", line 39, in load_image
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3227, in 

In [ ]:
task_name = '[11]objecttobackgroundobject'
mllms = ['seed', 'emu', 'gill']
shots = ['1', '2', '4']
theta_list = ['beach', 'street', 'park', 'forest', 'office', 'classroom', 'gym', 'library']
x_list = ['dog', 'cat', 'robot', 'car', 'chair', 'monkey', 'man', 'woman']
prompt2 = 'What is the background in this image? Answer from the following options: (1)beach (2)street (3)park (4)forest (5)office (6)classroom (7)gym (8)library'
prompt1 = 'What is the object in this image? Answer from the following options: (1)dog (2)cat (3)robot (4)car (5)chair (6)monkey (7)man (8)woman'

def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/background_object/' + theta + '_object'

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-be7061662205>", line 12, in <cell line: 12>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/background_object/beach_object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  

In [ ]:
task_name = '[1]colortocolorobject'
mllms = ['gill']
shots = ['1', '2', '4']
x_list = ['black', 'blue', 'green', 'pink', 'purple', 'red', 'white', 'yellow']
theta_list = ['flower', 'leaf', 'box', 'building', 'cup', 'bag', 'chair', 'car']
prompt1 = 'What is the object\'s color in this image? Answer from the following options: (1)black (2)blue (3)green (4)pink (5)purple (6)red (7)white (8)yellow'
prompt2 = 'What is the object in this image? Answer from the following options: (1)flower (2)leaf (3)box (4)building (5)cup (6)bag (7)chair (8)car'
def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/color_object/color_' + theta

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-76c54dac1631>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/color_object/color_flower'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

In [ ]:
task_name = '[2]objecttocolorobject'
mllms = ['gill']
shots = ['1', '2', '4']
theta_list = ['black', 'blue', 'green', 'pink', 'purple', 'red', 'white', 'yellow']
x_list = ['flower', 'leaf', 'box', 'building', 'cup', 'bag', 'chair', 'car']
prompt2 = 'What is the object\'s color in this image? Answer from the following options: (1)black (2)blue (3)green (4)pink (5)purple (6)red (7)white (8)yellow'
prompt1 = 'What is the object in this image? Answer from the following options: (1)flower (2)leaf (3)box (4)building (5)cup (6)bag (7)chair (8)car'
def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/color_object/' + theta + '_object'

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-b4a4887cd186>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/color_object/black_object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-b4a4887cd186>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/color_object/black_object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

In [ ]:
task_name = '[4]objecttocountobject'
mllms = ['gill']
shots = ['1', '2', '4']
theta_list = ['one', 'two', 'three', 'four']
x_list = ['person', 'cat', 'apple', 'chair', 'cup', 'dog', 'lion', 'shampoo']
prompt2 = 'How many objects are there in this image? Answer from the following options: (1)one (2)two (3)three (4)four'
prompt1 = 'What is the object in this image? Answer from the following options: (1)person (2)cat (3)apple (4)chair (5)cup (6)dog (7)lion (8)shampoo'
def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/count_object/' + theta + '_object'

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-28672bc3d42e>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/count_object/one_object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3575, in run_code
    self.showtraceback(running_compiled_code=True)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2101, in showtraceback
    stb = self.InteractiveTB.structured_traceback(etype,
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line

In [ ]:
task_name = '[3]counttocountobject'
mllms = ['gill']
shots = ['1', '2']
x_list = ['one', 'two', 'three', 'four']
theta_list = ['person', 'cat', 'apple', 'chair', 'cup', 'dog', 'lion', 'shampoo']
prompt1 = 'How many objects are there in this image? Answer from the following options: (1)one (2)two (3)three (4)four'
prompt2 = 'What is the object in this image? Answer from the following options: (1)person (2)cat (3)apple (4)chair (5)cup (6)dog (7)lion (8)shampoo'
def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/count_object/count_' + theta

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-02de69d80f86>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/count_object/count_person'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-02de69d80f86>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/count_object/count_person'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-02de69d80f86>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/count_object/count_person'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 

In [ ]:
task_name = '[5]styletostyleobject'
mllms = ['gill']
shots = ['1', '2', '4']
x_list = ['cartoon', 'oil', 'sketch', 'cubism', 'watercolor', 'origami']
theta_list = ['man', 'cat', 'flower', 'apple', 'dog', 'house', 'car', 'chair']
prompt1 = 'What is the style of this image? Answer from the following options: (1)cartoon (2)oil (3)sketch (4)cubism (5)watercolor (6)origami'
prompt2 = 'What is the object in this image? Answer from the following options: (1)man (2)cat (3)flower (4)apple (5)dog (6)house (7)car (8)chair'
def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/style_object/style_' + theta

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-8a77f3f791f2>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/style_object/style_man'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/u

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-8a77f3f791f2>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/style_object/style_man'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/u

In [ ]:
task_name = '[6]objecttostyleobject'
mllms = ['gill']
shots = ['1', '2', '4']
theta_list = ['cartoon', 'oil', 'sketch', 'cubism', 'watercolor', 'origami']
x_list = ['man', 'cat', 'flower', 'apple', 'dog', 'house', 'car', 'chair']
prompt2 = 'What is the style of this image? Answer from the following options: (1)cartoon (2)oil (3)sketch (4)cubism (5)watercolor (6)origami'
prompt1 = 'What is the object in this image? Answer from the following options: (1)man (2)cat (3)flower (4)apple (5)dog (6)house (7)car (8)chair'
def get_path(mllm, shot, theta):
    return './' + mllm + '_results/shot_' + shot + '_m/style_object/' + theta + '_object'

analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-5b4555c53fd7>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/style_object/cartoon_object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-5b4555c53fd7>", line 11, in <cell line: 11>
    analysis(task_name, mllms, shots, theta_list, x_list, prompt1, prompt2, get_path)
  File "<ipython-input-6-df3fe18c6c67>", line 16, in analysis
    for filename in os.listdir(cur_path):
OSError: [Errno 107] Transport endpoint is not connected: './emu_results/shot_1_m/style_object/cartoon_object'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil